# 02. Limpieza de datos

Este notebook contiene la limpieza del dataset de víctimas del conflicto armado en Colombia.

## Importación de Librerías

Importamos las librerías necesarias para el análisis de datos.

In [1]:
import pandas as pd
import csv
import os

## Carga del Dataset

Cargamos el dataset de víctimas del conflicto armado desde el archivo CSV.

In [ ]:
# Define the relative path to the data file
data_path = '../data/raw/victimas_por_hechos_departamental_20250416.csv'

# Load the dataset
try:
    # Attempt to read with UTF-8 encoding first
    df = pd.read_csv(data_path, encoding='utf-8') 
except UnicodeDecodeError:
    try:
        # Fallback to Latin-1 encoding if UTF-8 fails
        df = pd.read_csv(data_path, encoding='latin1') 
    except Exception as e:
        print(f"Error loading CSV file: {e}")
        df = None # Indicate failure by setting df to None

# Verificar si el dataset se cargó correctamente
if df is not None:
    print(f"Dataset cargado exitosamente. Dimensiones: {df.shape}")
else:
    print("Error al cargar el dataset. Por favor, verifica la ruta y la codificación del archivo.")

Dataset cargado exitosamente. Dimensiones: (2431164, 17)


# Eliminacion de columnas innecesarias
Se eliminaran las columnas que no sean necesarias para el analisis como NOM_RPT, COD_PAI y PAIS

In [4]:
# Mostrar forma inicial del dataframe y vista previa
print(f"Forma inicial del dataframe: {df.shape}")
print(f"Columnas del dataframe: {df.columns.tolist()}")

Forma inicial del dataframe: (2431164, 17)
Columnas del dataframe: ['FECHA_CORTE', 'NOM_RPT', 'COD_PAIS', 'PAIS', 'COD_ESTADO_DEPTO', 'ESTADO_DEPTO', 'PARAM_HECHO', 'HECHO', 'SEXO', 'ETNIA', 'DISCAPACIDAD', 'CICLO_VITAL', 'PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']


In [11]:
# Columnas que se eliminarán
columns_to_drop = ["NOM_RPT", "COD_PAIS", "PAIS", "FECHA_CORTE"]

# Eliminar columnas especificadas
df = df.drop(columns=columns_to_drop)

In [15]:
# Mostrar forma final del dataframe
print(f"Forma final del dataframe después de eliminar columnas: {df.shape}")
print(f"Columnas del dataframe: {df.columns.tolist()}")
df.head()

Forma final del dataframe después de eliminar columnas: (35250, 10)
Columnas del dataframe: ['HECHO', 'ESTADO_DEPTO', 'SEXO', 'CICLO_VITAL', 'ETNIA', 'PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']


,HECHO,ESTADO_DEPTO,SEXO,CICLO_VITAL,ETNIA,PER_OCU,PER_DECLA,PER_UBIC,PER_SA,EVENTOS
0,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 18 y 28,Ninguna,0.0,0.0,9.0,9.0,0.0
1,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 29 y 59,Ninguna,0.0,0.0,9.0,9.0,0.0
2,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 29 y 60,Ninguna,0.0,0.0,39.0,39.0,0.0
3,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 60 y 110,Ninguna,0.0,0.0,3.0,3.0,0.0
4,Abandono o Despojo Forzado de Tierras,Amazonas,Mujer,entre 29 y 59,Ninguna,0.0,0.0,7.0,7.0,0.0


# Corrección de errores de codificación y unificación de categorías

Se identificaron y reemplazaron valores con errores de codificación en varias columnas, con el fin de mejorar la calidad y consistencia de los datos. A continuación, se detallan los ajustes realizados:

### Columna: `ESTADO_DEPTO`

Valores corregidos:

* `"Nari�o"`
* `"NariÃ±o"`

Reemplazados por:

* `"Nariño"`

---

### Columna: `HECHO`

Valores corregidos:

* `"Desaparici�n forzada"`
* `"DesapariciÃ³n forzada"`

Reemplazados por:

* `"Desaparición forzada"`

---

* `"VinculaciÃ³n de NiÃ±os NiÃ±as y Adolescentes a Actividades Relacionadas con grupos armados"`
* `"Vinculaci�n de Ni�os Ni�as y Adolescentes a Actividades Relacionadas con grupos armados"`

Reemplazados por:

* `"Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados"`

---

* `"Minas Antipersonal, Munici�n sin Explotar y Artefacto Explosivo improvisado"`

Reemplazado por:

* `"Minas Antipersonal, Munición sin Explotar y Artefacto Explosivo improvisado"`

---

### Columna: `ETNIA`

Unificación de categorías para facilitar el análisis:

Valores originales:

* `"Negro (Acreditado RA)"`
* `"Afrocolombiano (Acreditado RA)"`

Reemplazados por:

* `"Negro(a) o Afrocolombiano(a) (Acreditado RA)"`


In [13]:
# Corrección de errores de codificación

df['ESTADO_DEPTO'] = df['ESTADO_DEPTO'].replace({
    'Nari�o': 'Nariño',
    'NariÃ±o': 'Nariño'
})

df['HECHO'] = df['HECHO'].replace({
    'Desaparici�n forzada': 'Desaparición forzada',
    'DesapariciÃ³n forzada': 'Desaparición forzada',
    'VinculaciÃ³n de NiÃ±os NiÃ±as y Adolescentes a Actividades Relacionadas con grupos armados':
        'Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados',
    'Vinculaci�n de Ni�os Ni�as y Adolescentes a Actividades Relacionadas con grupos armados':
        'Vinculación de Niños Niñas y Adolescentes a Actividades Relacionadas con grupos armados',
    'Minas Antipersonal, Munici�n sin Explotar y Artefacto Explosivo improvisado':
        'Minas Antipersonal, Munición sin Explotar y Artefacto Explosivo improvisado'
})

df['ETNIA'] = df['ETNIA'].replace({
    'Negro (Acreditado RA)': 'Negro(a) o Afrocolombiano(a) (Acreditado RA)',
    'Afrocolombiano (Acreditado RA)': 'Negro(a) o Afrocolombiano(a) (Acreditado RA)'
})

df['CICLO_VITAL'] = df['CICLO_VITAL'].replace({
'entre 61 y 100': 'entre 60 y 110'
})


# Limpieza y Validación de Grupos Únicos para Reemplazo de NaN por Cero

Este análisis tiene como objetivo **justificar y ejecutar** el reemplazo de valores faltantes (`NaN`) por `0` en las variables de conteo:

- `PER_OCU`
- `PER_DECLA`
- `PER_UBIC`
- `PER_SA`
- `EVENTOS`

El procedimiento se realiza **únicamente si se confirma que no existen registros duplicados** por grupo definido a partir de:

- `FECHA_CORTE`
- `HECHO`
- `ESTADO_DEPTO`
- `SEXO`
- `CICLO_VITAL`
- `ETNIA`

De esta forma se garantiza que cada fila representa un grupo único y que los valores nulos pueden interpretarse como **"cero personas/eventos registrados"**.



In [14]:
# Paso 1: Definir columnas
columnas_grupo = ['HECHO', 'ESTADO_DEPTO', 'SEXO', 'CICLO_VITAL', 'ETNIA']
variables_conteo = ['PER_OCU', 'PER_DECLA', 'PER_UBIC', 'PER_SA', 'EVENTOS']

# Paso 2: Agrupar y sumar valores de conteo
df = df.groupby(columnas_grupo, as_index=False)[variables_conteo].sum()

df

,HECHO,ESTADO_DEPTO,SEXO,CICLO_VITAL,ETNIA,PER_OCU,PER_DECLA,PER_UBIC,PER_SA,EVENTOS
0,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 18 y 28,Ninguna,0.0,0.0,9.0,9.0,0.0
1,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 29 y 59,Ninguna,0.0,0.0,9.0,9.0,0.0
2,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 29 y 60,Ninguna,0.0,0.0,39.0,39.0,0.0
3,Abandono o Despojo Forzado de Tierras,Amazonas,Hombre,entre 60 y 110,Ninguna,0.0,0.0,3.0,3.0,0.0
4,Abandono o Despojo Forzado de Tierras,Amazonas,Mujer,entre 29 y 59,Ninguna,0.0,0.0,7.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...
35245,Vinculación de Niños Niñas y Adolescentes a Ac...,Vichada,Mujer,entre 29 y 59,Ninguna,126.0,14.0,7.0,7.0,140.0
35246,Vinculación de Niños Niñas y Adolescentes a Ac...,Vichada,Mujer,entre 29 y 60,Indigena,65.0,0.0,0.0,0.0,65.0
35247,Vinculación de Niños Niñas y Adolescentes a Ac...,Vichada,Mujer,entre 29 y 60,Indigena (Acreditado RA),38.0,0.0,0.0,0.0,38.0
35248,Vinculación de Niños Niñas y Adolescentes a Ac...,Vichada,Mujer,entre 29 y 60,Negro(a) o Afrocolombiano(a),34.0,0.0,0.0,0.0,34.0


In [46]:
# Paso 3: Reemplazar NaN por 0
df[variables_conteo] = df[variables_conteo].fillna(0)

In [47]:
# Paso 4: Validar que no queden NaN
print("¿Quedan NaN después del reemplazo?")
print(df[variables_conteo].isna().sum())

¿Quedan NaN después del reemplazo?
PER_OCU      0
PER_DECLA    0
PER_UBIC     0
PER_SA       0
EVENTOS      0
dtype: int64


# Exportamos los datos limpios a un archivo CSV
Se exporta el archivo CSV con los datos limpios en en ´/data/processed/victimas_por_hechos_departamental_20250416.csv´ para su posterior análisis

In [50]:
df.to_csv(
    path_or_buf='../data/processed/victimas_por_hechos_departamental_20250416.csv',
    sep=',',
    na_rep='',
    header=True,
    index=False,
    encoding='utf-8',
    quoting=csv.QUOTE_MINIMAL,
    lineterminator=os.linesep,
    quotechar='"',
    decimal='.',
    errors='strict'
)